In [3]:
import numpy as np
import glob
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from tqdm import tqdm

import cv2
from utils.utils_lcn import infer_box, camera_to_image_frame
from utils.utils_hpe import optimize_scaling_factor

In [4]:
files = glob.glob("D:\\github\\FS-Jump3D\\data\\npy\\**\\*.npy", recursive=True)
output_dir = "D:\\github\\MotionAGFormer\\data\\keypoints"

In [5]:
for file in tqdm(files):
    parts = file.split(os.sep)
    skater = parts[-3].lower()
    filename = os.path.splitext(parts[-1])[0].lower()

    data = np.load(file)
    data[:, :, [1, 2]] = data[:, :, [2, 1]]  # Swap Y and Z
    data[:, :, 1] = -data[:, :, 1]  # Flip Y axis

    for cam_id in range(12):
        new_name = f"{skater}_cam_{cam_id+1}_{filename}_3D.npy"
        np.save(os.path.join(output_dir, new_name), data)

100%|██████████| 253/253 [00:01<00:00, 134.53it/s]


## Generate PKL

In [6]:
motion2d_files = glob.glob("../keypoints/*_2D.npy", recursive=False)
motion3d_files = glob.glob("../keypoints/*_3D.npy", recursive=False)

In [7]:
def get_camera_pose(world_pts, img_pts, camera):
    fx = camera["fx"]
    fy = camera["fy"]
    cx = camera["cx"]
    cy = camera["cy"]

    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=np.float32)

    dist = np.zeros(4, dtype=np.float32)

    success, rvec, tvec = cv2.solvePnP(
        world_pts.astype(np.float32), img_pts.astype(np.float32), K, dist
    )

    if success:
        R, _ = cv2.Rodrigues(rvec)
        T = tvec.flatten()
        return R, T
    return None, None


def world_to_cam(world_pose, R, T):
    return np.dot(world_pose, R.T) + T


In [8]:
def load_db(pose_3ds, pose_2ds, camera):
    pose_2ds[:, 0] = (pose_2ds[:, 0] * camera["cx"] * 2).astype(int)
    pose_2ds[:, 1] = (pose_2ds[:, 1] * camera["cy"] * 2).astype(int)

    data = {}
    for key in [
        "pose_2d",
        "pose_3d",
        "cam_3d",
        "image_3d",
        "image_2.5d",
        "2.5d_factor",
    ]:
        data[key] = []
    R, T = get_camera_pose(pose_3ds[0], pose_2ds[0][:, :2], camera)
    for pose_2d, pose_3d in zip(pose_2ds, pose_3ds):
        cam_3d = world_to_cam(pose_3d, R, T) * 1000
        cam_3d_hat = cam_3d - cam_3d[0]

        box = infer_box(pose3d=cam_3d, camera=camera, rootIdx=0)
        img_3d = camera_to_image_frame(cam_3d, box, camera, rootIdx=0)
        img_3d_hat = img_3d - img_3d[0]

        #pred_lambda, losses = optimize_scaling_factor(img_3d_hat, cam_3d_hat)

        #img_25d = img_3d * pred_lambda

        data["pose_2d"].append(np.array(pose_2d).copy())
        data["pose_3d"].append(np.array(pose_3d).copy())
        data["cam_3d"].append(np.array(cam_3d).copy())
        data["image_3d"].append(np.array(img_3d).copy())
        #data["image_2.5d"].append(np.array(img_25d).copy())
        #data["2.5d_factor"].append(np.array(pred_lambda).copy())

    return data, np.array(pose_2d), np.array(img_3d)

In [11]:
import pickle
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


def process_file(file_2d):
    file_3d = file_2d.replace("_2D.npy", "_3D.npy")

    # Check if pickle file already exists
    filename = file_3d.replace("_3D.npy", ".pkl").replace("../keypoints\\", "")

    # if os.path.exists(os.path.join(output_dir, filename)):
    #    return

    fx, fy = 1000, 1000
    cx, cy = 960.0, 540.0
    camera = {"fx": fx, "fy": fy, "cx": cx, "cy": cy}

    if file_3d in motion3d_files:
        pose_2ds = np.load(file_2d)
        pose_3ds = np.load(file_3d)

        data, _, _ = load_db(pose_3ds, pose_2ds, camera)

        with open(os.path.join(output_dir, filename), "wb") as f:
            pickle.dump(data, f)


# Multi-threaded execution
max_workers = 4  # Adjust based on your system
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    futures = [executor.submit(process_file, file_2d) for file_2d in motion2d_files]

    # Process completed tasks with progress bar
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"Error processing file: {e}")

100%|██████████| 3036/3036 [02:04<00:00, 24.33it/s]


In [10]:
def convert_3dnpy():
    for file_2d in tqdm(motion2d_files):
        file_3d = file_2d.replace("_2D.npy", "_3D.npy")

        fx, fy = 650, 650
        cx, cy = 960.0, 540.0
        camera = {"fx": fx, "fy": fy, "cx": cx, "cy": cy}

        if file_3d in motion3d_files:
            pose_2ds = np.load(file_2d)
            pose_3ds = np.load(file_3d)

            data, _, motion_3d = load_db(pose_3ds, pose_2ds, camera)
            
            np.save(os.path.join(output_dir, file_3d), motion_3d)